<a href="https://colab.research.google.com/github/hrstbangera/Forecasting-Time-Series-Data-with-Facebook-Prophet/blob/main/finetune_falcon_7b_sharded_freeGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine-tune Falcon-7b-instruct-sharded model** on a mental health conversational dataset curated by heliosbrahma can be found on Hugging Face.
Links to both the model and dataset are in the notebook.


##Installs and imports

In [1]:
#all installs
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb
!pip install huggingface_hub

#all imports
import torch
import time
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from transformers import TrainingArguments
from trl import SFTTrainer

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196

##Notebook connection to Hugging face

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
notebook_login()

##Loading the dataset from hugging face

In [4]:
dataset_name = "heliosbrahma/mental_health_chatbot_dataset"
data = load_dataset(dataset_name)
data

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

##Loading the model and Setting up bitsandbytes config

We will use sharded version of falcon-7b-instruct model



In [5]:
model_name = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
model.config.use_cache = False

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

configuration_falcon.py:   0%|          | 0.00/6.70k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_falcon.py:   0%|          | 0.00/56.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vilsonrodrigues/falcon-7b-instruct-sharded:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00013-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/828M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

##Loading the tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

##Setting up the LoRA config

In [7]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 #16
lora_dropout = 0.05 #0.1
lora_rank = 32 #64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


##Load the trainer

In [9]:
output_dir = "falcon7binstruct_mentalhealthmodel_oct23"
per_device_train_batch_size = 16 #4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 100 #100 #500
warmup_ratio = 0.03
lr_scheduler_type = "cosine" #"constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True
)

##Passing arguments to the SFTT trainer

In [10]:
max_seq_length = 256

trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [11]:
# upcasting the layer norms in torch.bfloat16 for more stable training
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.bfloat16)

##Train the model

You can check your training time if you are doing multiple experiments

In [12]:
start = time.time()

In [13]:
peft_model.config.use_cache = False
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
10,1.656800
20,1.301300
30,0.912100
40,0.459900
50,0.170700
60,0.066200
70,0.045700
80,0.040500
90,0.038800
100,0.038000


TrainOutput(global_step=100, training_loss=0.4729997545480728, metrics={'train_runtime': 660.6616, 'train_samples_per_second': 9.687, 'train_steps_per_second': 0.151, 'total_flos': 5.302835892765082e+16, 'train_loss': 0.4729997545480728, 'epoch': 36.36})

In [14]:
end=time.time()

In [15]:
time_taken=end-start
print(time_taken)

675.8279531002045


##Save the model

In [16]:
#trainer.save() #if you want to save your model locally

##Push to hub

In [17]:
trainer.push_to_hub()

events.out.tfevents.1706003098.cc1f1b0dfb91.1322.0:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HarshithNLP/falcon7binstruct_mentalhealthmodel_oct23/commit/1fafaeef660e2e597e492a01fccb1b63da9cf8c6', commit_message='End of training', commit_description='', oid='1fafaeef660e2e597e492a01fccb1b63da9cf8c6', pr_url=None, pr_revision=None, pr_num=None)

##Inference

In [18]:
# Loading PEFT model
PEFT_MODEL = "HarshithNLP/falcon7binstruct_mentalhealthmodel_oct23"
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [21]:
# Generate responses from both orignal model and fine-tuned model
def get_response(question):
  prompt = f"""
  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: {question}

  ###Response:

  """

  # encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
  # outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
  #                                                                                                                    eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
  #                                                                                                                    temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  # text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  # #print(dashline)
  # print(f'Response from original falcon_7b_instruct_sharded:\n{text_output}')

  print("*******************************************************")

  peft_encoding = peft_tokenizer(prompt, return_tensors="pt").to("cuda:0")
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                      eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                      temperature=0.1, top_p=0.1, repetition_penalty=1.2, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'Response from fine-tuned falcon_7b_instruct_sharded:\n{peft_text_output}')


In [22]:
get_response("Are there cures for mental health problems?")

*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are there cures for mental health problems?

  ###Response:

  **While the majority of mental health conditions can be effectively managed with treatments like medication, therapy, and lifestyle changes, the response that many professionals and individuals agree on is:**

**It depends on the individual.**
Some mental health conditions may have complete remission with the right treatment, while others might persist into long-term management. Many factors influence the nature and duration of specific mental health disorders, including genetic predisposition, life experiences, and personal resilience. Therefore, seeking a "cure" might not always be possible for all cases.

However

In [23]:
get_response("What’s the difference between psychotherapy and counselling?")

*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What’s the difference between psychotherapy and counselling?

  ###Response:

   Psychotherapy and counselling are terms that refer to the same types of treatments as they both focus on providing support and guidance to individuals and families. The primary distinction between them is their scope. Counseling tends to be more short-term and focused, with a specific goal of helping the individual to deal with a particular problem (e.g., quitting smoking, managing stress, etc.). Therapy can be either cognitive-behavioral or psychodynamic. Cognitive-behavioral therapy helps individuals change negative thought patterns and behaviors, while psychotherapy includes behavioral changes a

In [24]:
get_response("What happens in a therapy session?")

*******************************************************
Response from fine-tuned falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: What happens in a therapy session?

  ###Response:

   Therapy is a supportive and confidential process between you and your therapist. At therapy sessions, there isn't much that requires any specific knowledge or expertise on your part. The therapeutic approach is collaborative between you and your therapist. Your therapist will tailor the treatment to suit your unique needs, and you'll work together to achieve your goals. To make the therapy experience comfortable and effective, you might share your thoughts and feelings, either verbally or in writing. Your therapist will listen actively and provide guidance or suggestions to help you gain insights and navigate through challe

In [ ]:
get_response("Are neurofeedback and biofeedback the same thing?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question: Are neurofeedback and biofeedback the same thing?

  ###Response:

  ###Explanation:
  Neurofeedback and biofeedback are similar in that they both involve the use of technology to help individuals learn how to control their body and mind. However, there are some differences between the two. Neurofeedback is typically used to help individuals learn how to control their brain waves, while biofeedback is used to learn how to control other bodily functions, such as heart rate or blood pressure. Additionally, neurofeedback is often used to treat conditions such as anxiety, depression, and ADHD, while biofeedback is often used to treat conditions such as chronic pain and migraines.
*******************************************************
Res

In [ ]:
get_response(" What is insomnia disorder?")

Response from original falcon_7b_instruct_sharded:

  ###Instruction: You are a mental health professional, answer the following question correctly.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'

  ###Question:  What is insomnia disorder?

  ###Response:

  ###Explanation: Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain medications. It can also lead to other health problems if left untreated. It is important to seek medical attention if you suspect you have Insomnia disorder.

Incorrect response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep.'

Correct response: 'Insomnia disorder is a sleep disorder where a person has difficulty falling asleep or staying asleep. It can be caused by a variety of factors, including anxiety, depression, and certain med